# 1. Работаем с извлечением данных

In [1]:
# Импортируем библиотеку для работы с Postgres на питоне
import psycopg2

# Пытаемся подключиться к БД, при ошибке выскакивает alert
try:
    # Создаем подключение к БД
    conn = psycopg2.connect(dbname='postgres', user='postgres', password='123', host='localhost')
    # Создаем объект курсора для подключенной БД
    cursor = conn.cursor()
except:
    print('Не удалось подключиться к БД!')

In [2]:
# Запросом извлекаем данные из БД
cursor.execute('SELECT name, description, salary, region_name FROM jobs.joblist')
# Сохраняем и предобрабатываем данные
all_info = list(map(list, cursor.fetchall()))
# Запросом извлекаем данные из БД
cursor.execute('SELECT main_skills FROM jobs.joblist')
# Сохраняем и предобрабатываем данные
all_skills = list(set(sum([i[0].split(';') for i in list(map(list, cursor.fetchall()))], [])))
# Удаляем лишний элемент из массива
del all_skills[0]

In [3]:
# Закрываем объект курсора
cursor.close()
# Закрываем соединение с БД
conn.close()

# 2. Работаем с преодбработкой данных

In [4]:
# Импортиурем библиотеку для леметизации слов
import spacy
# Импортируем библиотеку для работы со строками
import re

# Загружаем предобученную модель для леметизации русских слов
nlp = spacy.load('ru_core_news_sm')

# Циклом проходимся по всем описаниям
for i in range(len(all_info)):
    # Сохраняем описание и удаляем из него лишнии знаки
    description = re.sub(r'[—\'"(){}[\]/.,?!-_:;]', r'', all_info[i][1])
    # Получаем лемитизированные описание
    doc = nlp(description)
    # Заменяем старое описание на новое предобработаннео
    all_info[i][1] = " ".join([token.lemma_ for token in doc])

# 3. Работаем с NLP алгоритмами для данных

In [5]:
# Импортируем библиотеку для работы с нейросетями
import torch
# Из библиотеки импортируем модули модели BERT и токенайзера для нее
from transformers import BertModel, BertTokenizer
# Из библиотеки импортируем реализацию классификатора к ближайших соседей
from sklearn.neighbors import KNeighborsClassifier

# Загружем токенайзер для модели Bert
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
# Загружаем модель типа Bert
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')

# Создаем словарь для обучения модели
d = {i: 'Навык' for i in all_skills} | {i: 'Не навык' for i in ['предоставляем', 'банкам', 'region', 'приглашаем', 'company', 'компания', 'маркетинговых', 'потоков', 'компанией']}

# Сохраняем признаки X для обучения
words = list(d.keys())

# Создаем массив для сохранения векторных представлений слов
word_embeddings = []
# Циклом проходимся по всем слова
for word in words:
    # Токенезиурем слово и преобразуем в тензор
    inputs = tokenizer(word, return_tensors='pt', max_length=128 ,truncation=True)
    # Запрещаем torch использовать градиентный спуск
    with torch.no_grad():
        # Получаем выходной слой модели
        outputs = model(**inputs)
        # Усредняем по токенам
        word_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        # Добавляем в ранее созданный массив
        word_embeddings.append(word_embedding)

# Создаем алгоритм KNN
knn_classifier = KNeighborsClassifier(n_neighbors=1)
# Фитим нашу модель
knn_classifier.fit(word_embeddings, list(d.values()))

# Тестово предиктим 
predicted_classes = knn_classifier.predict(word_embeddings)

# Создаем счетчик
count = 0
# Выводим пример работы модели
for word, predicted_class in zip(words, predicted_classes):
    print(f'{word} -> {predicted_class}')
    if count == 5:
        break
    count += 1

c:\Users\pupki\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of 

apache superset -> Навык
математический анализ -> Навык
знания в области логистики -> Навык
имитационное моделирование -> Навык
power query -> Навык
разработка проектной документации -> Навык


In [6]:
# Создаем массив со всеми описаниями
texts = [i[1] for i in all_info]

# Создаем массив для хранения в нем классов для каждого предложения
predicted_classes_per_sentences = []

# Циклом проходимся по всем данным
for text in texts:
    # Создаем из текста массив слов
    words = text.split()
    # Создаем массив для хранения в нем векторных представлений слов
    word_embeddings = []
    # Циклом проходимся по всем словам
    for word in words:
        # Токенезиурем слово и преобразуем в тензор
        inputs = tokenizer(word, return_tensors='pt', max_length=128 ,truncation=True)
        # Запрещаем torch использовать градиентный спуск
        with torch.no_grad():
            # Получаем выходной слой модели
            outputs = model(**inputs)
            # Усредняем по токенам
            word_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
            # Добавляем в ранее созданный массив
            word_embeddings.append(word_embedding)

    # Предиктим классы
    predicted_classes = knn_classifier.predict(word_embeddings)
    # Добавляем полученные классы в ранее созданный массив
    predicted_classes_per_sentences.append(predicted_classes)


# Создаем массивы для всех вакансий, где будем хранить все обнаруженные навыки
for i in range(len(all_info)):
    all_info[i].append([])

# Создаем счетчик
count = 0
# Прохожимся по всем данным
for text, predicted_classes in zip(texts, predicted_classes_per_sentences):
    # Из текста получаем массив слов
    words = text.split()
    # Циклом проходимся по всем словам и их классам
    for word, predicted_class in zip(words, predicted_classes):
        # Тестово показываем работу модели
        if count < 10:
            print(f'{word} -> {predicted_class}')
        # Проверяем условие, если класс == "Навык" и длинна слова больше 2-х, то добавляем в массив
        if predicted_class == 'Навык' and len(word) > 2:
            all_info[count][4].append(word)

    # Удаляем из массива дубликаты навыков
    all_info[count][4] = list(set(all_info[count][4]))
    # Увеличиваем счетчик
    count += 1

In [7]:
# Из библиотеки импортируем модуль для работы с кластерами
from sklearn.cluster import KMeans

# Создаем массив для сохранения всех навыков в одном месте
all_detected_skills = []
# Циклом проходимся по всем данным
for i in range(len(all_info)):
    all_detected_skills += all_info[i][4]

# Задаем максимальную длину последовательности
max_length = 32

# Векторизация слов
encoded_texts = [tokenizer(word, return_tensors='pt', padding='max_length', max_length=max_length, truncation=True) for word in all_detected_skills]
input_ids = torch.cat([encoded_text['input_ids'] for encoded_text in encoded_texts], dim=0)
attention_masks = torch.cat([encoded_text['attention_mask'] for encoded_text in encoded_texts], dim=0)

# Получаем эмбеддингов слов с помощью Bert
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_masks)
    word_embeddings = outputs.last_hidden_state[:, 0, :].numpy()

# Задаем количество кластеров
num_clusters = 3
# Создаем алгоритм KMeans
kmeans = KMeans(n_clusters=num_clusters)
# Фитим нашу модель
kmeans.fit(word_embeddings)

# Создаем многомерный массив для сохранения навыков
cluster_skills = [[] for _ in range(num_clusters)]

# Заполняем созданный массив навыками
for skill, cluster_label in zip(all_detected_skills, kmeans.labels_):
    cluster_skills[cluster_label].append(skill)


c:\Users\pupki\anaconda3\envs\nlp\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


# 4. Работаем с анализом данных

- #### 4.1 Определяем наиболее востребованные и наименее востребованные навыки для выбранной группы вакансий

In [8]:
# Из библиотеки имопртируем модуль счетчика
from collections import Counter

# Считаем вхождение всех навыков 
skill_counter = Counter(all_detected_skills)

# Выводим наиболее востребованные навыки
print(f'Наиболее востребованные навыки: {skill_counter.most_common(10)}', end='\n\n')
# Выводим наименее востребованные навыки
print(f'Наименее востребованные навыки: {skill_counter.most_common()[-10:]}')

Наиболее востребованные навыки: [('знание', 67), ('sql', 59), ('быть', 57), ('график', 47), ('автоматизация', 37), ('рост', 35), ('рынок', 34), ('предлагать', 33), ('python', 30), ('excel', 28)]

Наименее востребованные навыки: [('curl', 1), ('kubernetes', 1), ('финтехopen', 1), ('нефункциональный', 1), ('опцион', 1), ('dod', 1), ('бэклога', 1), ('story', 1), ('uml', 1), ('ddd', 1)]


- #### 4.2 Определяем наиболее оплачеваемые навыки

In [9]:
# Создаем массив, куда сохраним наиболее оплачиваемые навыки
most_paid_skills = []
# Находим среднюю зарплату
avg_salary = sum([i[2] for i in all_info]) / len([i[2] for i in all_info])
# Проходимся по всем данным
for i in range(len(all_info)):
    # Проверяем условие, если зарплата вакансии в два раза выше, чем средняя зарплата, то добавляем в массив
    if all_info[i][2] > avg_salary * 2:
        most_paid_skills += all_info[i][4]

# Убираем дубликаты навыков
most_paid_skills = list(set(most_paid_skills))
# Выводим результат
print(f'Наиболее оплачеваемые навыки: {most_paid_skills}')

Наиболее оплачеваемые навыки: ['brands', 'дать', 'branch', 'mesh', 'разработкуаналитический', 'воркшопы', 'тесткейсов', 'appmetrica', 'понастоящему', 'ios', 'skyeng', 'deep', 'sql', 'analytics', 'аналитикааналитика', 'насколько', 'eltetlинструмент', 'тпбудет', 'предлагать', 'знание', 'приготовить', 'postgre', 'заниматьсяразработка', 'debezium', 'развивать', 'данныхумение', 'мэппингов', 'крутой', 'математик', 'azure', 'cicd', 'стек', 'google', 'загружать', 'счёт', 'доработка', 'superset’а', 'дмс', 'удаленныйофисныйгибридный', 'быть', 'попап', 'iceberg', 'удаленкаоформление', 'оплата', 'курс', 'mppcloud', 'selfserve', 'математический', 'senior', 'данныхпроектирование', 'redshift', 'кхд', 'ценность', 'snowflake', 'winwin', 'mlops', 'настройка', 'повышение', 'предстоять', 'delta', 'оптимизацииизменении', 'тестированиеответственность', 'binom', 'пиццамейкерам', 'реклама', 'писать', 'dbt', 'твой', 'kafka', 'тулинга', 'перфекционизмом', 'маппингов', 'страныоформление', 'уметь', 'пайплайнов', 

- #### 4.3 Орпеделяем региональную специфику востребованных навыков

In [1]:
# Создаем словарь для наиболее востребованных навыков
most_needed_skills = {name: sum([i[4] for i in all_info if i[3] == name], []) for name in [i[3] for i in all_info]}
# Проходимся циклом по словарю
for region, skills in most_needed_skills.items():
    # Считаем вхождение навыков для региона
    skill_counter = Counter(skills)
    # Добавляем 5 наиболее востребованных навыков для региона
    most_needed_skills[region] = skill_counter.most_common(5)

# Выводим наиболее востребованные навыки для каждого региона
for region in most_needed_skills.keys():
    print(f'{region.capitalize()}: {most_needed_skills[region]}')

NameError: name 'all_info' is not defined

# 5. Работаем с сохранением данных в Excel

In [11]:
# Импортируем билиотеку для работы с Excel на питоне
import openpyxl

# Создаем массив для хранения данных
data_for_excel = [[] for _ in range(num_clusters)]
# Проходимся по всем кластерам
for i in range(num_clusters):
    # Считаем вхождение навыков в каждом кластере
    skill_counter = Counter(cluster_skills[i])
    # Сохраняем в ранее созданный массив
    data_for_excel[i] = skill_counter

# Создаем лист Excel
wb = openpyxl.Workbook()
# Получаем активный лист
sheet = wb.active

# Создаем заголовки для листа
headers = ['Навык', 'Количество', 'Кластер']
# Циклом заполняем заголовки на первой строке
for col, header in enumerate(headers, start=1):
    # Внутренним методом заполняем первую строку
    sheet.cell(row=1, column=col, value=header)

# Задаем индекс строки
row_index = 2

# Циклом проходимся по всем кластерам
for cluster_index, cluster in enumerate(data_for_excel, start=1):
    # Циклом заполняем строки
    for skill, count in cluster.items():
        sheet.cell(row=row_index, column=1, value=skill)
        sheet.cell(row=row_index, column=2, value=count)
        sheet.cell(row=row_index, column=3, value=f'Кластер {cluster_index}')
        # Увеличиваем индекс строки
        row_index += 1

# Сохраняем лист Excel
wb.save('mainskills.xlsx')

________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

###### Сохраняем данные с этого модуля, для заполнения графа связей на следующем модуле

In [12]:
# Сохраняем данные в массив
data = [i[4] for i in all_info]

# Открываем файл
with open('dataModuleB.txt', 'w') as f:
    # Циклом проходимся по данным
    for value in data:
        # Сохраняем данные в файл
        f.write(f'{value},\n')